In [33]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup
import time
import numpy as np
import lxml
import chardet
import datetime


In [4]:
def getLeagueTeamsUrls(ligaMxUrl):
    data = req.get(ligaMxUrl)
    #soup = BeautifulSoup(data.text, features = "html.parser")
    soup = BeautifulSoup(data.text, "lxml")

    standings_table=soup.select('table.stats_table')[0]

    #Retrieving team links
    links = standings_table.find_all('a')
    links =  [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]

    league_urls = [f"https://fbref.com{l}" for l in links]
    return league_urls, soup


In [5]:
def getMatches(team_url):
    team_web_data = req.get(team_url)
    team_matches = pd.read_html(team_web_data.text, match = "Scores & Fixtures")[0]
    return team_web_data, team_matches

In [6]:
def getShooting(team_data):
    #soup = BeautifulSoup(team_data.text, features = "html.parser")
    soup = BeautifulSoup(team_data.text, 'lxml')

    shooting_link = soup.find_all("a")
    shooting_link = [l.get("href") for l in shooting_link]
    shooting_link = [l for l in shooting_link if l and 'all_comps/shooting/' in l]

    shooting_data = req.get(f"https://fbref.com{shooting_link[0]}")
    shootingInfo = pd.read_html(shooting_data.text, match='Shooting')[0]

    #transformating shooting info
    shootingInfo.columns = shootingInfo.columns.droplevel()
    
    shootingNewColumns = ["Date","Sh","SoT", "Dist","FK","PK","PKatt"]
    for newColumn in shootingNewColumns:
        if newColumn not in shootingInfo.columns:
            shootingInfo[newColumn] = np.nan

    return shootingInfo



In [30]:
sample = pd.read_csv("matches23-17_2.csv").sort_values(by=['date'], ascending =False)
sample.loc[sample["team_name"] == "Guadalajara"]
max(sample["date"])
#sample["team_name"] == "Guadalajara"]
#sample[team_name=="Guadalajara"]

'2023-02-27'

In [46]:
def infoRetriever(baseUrl, years=[]):
    
    if years == []:
        today = datetime.date.today()
        year = today.strftime("%Y")
        years = [year]
        
    all_matches = []
    
    for year in years:
        league_urls, seasonSoup = getLeagueTeamsUrls(ligaMxUrl)

        # Find anchor element with class 'prev' which contains link to previous season
        previous_season = seasonSoup.select("a.prev")[0].get("href")
        baseUrl = f"https://fbref.com{previous_season}"

        for team_url in league_urls:
            team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
            #print('Retrieving info from team',team_name, 'in season', year)
            teamWebData, teamMatchesDf = getMatches(team_url)

            #print('Retrieving shooting info from team',team_name, 'in season', year)
            teamShootingDf = getShooting(teamWebData)
            #time.sleep(1)

            try:
                print('Merging datasets...')
                team_df = teamMatchesDf.merge(teamShootingDf[["Date","Sh","SoT", "Dist",'FK','PK',"PKatt"]], on = "Date")
            except ValueError:
                continue

            team_df["team_name"] = team_name
            team_df["season"] = "2023"
            all_matches.append(team_df)
            time.sleep(1)   

    league_matches = pd.concat(all_matches)
    league_matches.columns = [c.lower() for c in league_matches.columns] 
    
    return league_matches

       

In [50]:
def updateMatches(baseFile, newFile):    
    
    thisSeasonMatches = infoRetriever('https://fbref.com/en/comps/31/Liga-MX-Stats')
    
    '''
    ligaMxUrl = 'https://fbref.com/en/comps/31/Liga-MX-Stats'
    
    # List of dataframes: Dataframes with each team's matches information per season
    all_matches = []

    #Getting matches and shots Info
    league_urls, seasonSoup = getLeagueTeamsUrls(ligaMxUrl)

    for team_url in league_urls:
        team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
        #print('Retrieving info from team',team_name, 'in season', year)
        teamWebData, teamMatchesDf = getMatches(team_url)

        #print('Retrieving shooting info from team',team_name, 'in season', year)
        teamShootingDf = getShooting(teamWebData)
        #time.sleep(1)

        try:
            print('Merging datasets...')
            team_df = teamMatchesDf.merge(teamShootingDf[["Date","Sh","SoT", "Dist",'FK','PK',"PKatt"]], on = "Date")
        except ValueError:
            continue

        team_df["team_name"] = team_name
        team_df["season"] = "2023"
        all_matches.append(team_df)
        time.sleep(1)   
        
    thisSeasonMatches = pd.concat(all_matches)
    thisSeasonMatches.columns = [c.lower() for c in thisSeasonMatches.columns] 
    '''

    baseDf = pd.read_csv(baseFile)
    
    updatedDf = pd.concat([baseDf,thisSeasonMatches], ignore_index= True)
    updatedDf = updatedDf.drop_duplicates(subset = ["date","team_name"], keep='last', inplace = False)
    
    updatedDf.to_csv(newFile, index=False )
    
        

    #team_df.head()


In [51]:
updateMatches("matches23-17_2.csv","matches23-17_3.csv")

Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...
Merging datasets...


TypeError: concat() got an unexpected keyword argument 'ingnore_index'

In [9]:
ligaMxUrl = 'https://fbref.com/en/comps/31/Liga-MX-Stats'

In [25]:
#league_urls=['https://fbref.com/en/squads/c02b0f7a/Guadalajara-Stats']

years = list(range(2023, 2017, -1))
ligaMxUrl = 'https://fbref.com/en/comps/31/Liga-MX-Stats'
#years = [2023]

# List of dataframes: Dataframes with each team's matches information per season
all_matches = []

for year in years: 
    league_urls, seasonSoup = getLeagueTeamsUrls(ligaMxUrl)
    
    # Find anchor element with class 'prev' which contains link to previous season
    previous_season = seasonSoup.select("a.prev")[0].get("href")
    ligaMxUrl = f"https://fbref.com{previous_season}"

    for team_url in league_urls:
        team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
        print('Retrieving info from team',team_name, 'in season', year)
        teamWebData, teamMatchesDf = getMatches(team_url)
        
        print('Retrieving shooting info from team',team_name, 'in season', year)
        teamShootingDf = getShooting(teamWebData)
        #time.sleep(1)
        
        try:
            print('Merging datasets...')
            team_df = teamMatchesDf.merge(teamShootingDf[["Date","Sh","SoT", "Dist",'FK','PK',"PKatt"]], on = "Date")
        except ValueError:
            continue

        team_df["team_name"] = team_name
        team_df["season"] = year
        all_matches.append(team_df)
        time.sleep(1)   

#team_df.head()


Retrieving info from team Monterrey in season 2023
Retrieving shooting info from team Monterrey in season 2023
Merging datasets...
Retrieving info from team America in season 2023
Retrieving shooting info from team America in season 2023
Merging datasets...
Retrieving info from team UANL in season 2023
Retrieving shooting info from team UANL in season 2023
Merging datasets...
Retrieving info from team Pachuca in season 2023
Retrieving shooting info from team Pachuca in season 2023
Merging datasets...
Retrieving info from team Toluca in season 2023
Retrieving shooting info from team Toluca in season 2023
Merging datasets...
Retrieving info from team Santos Laguna in season 2023
Retrieving shooting info from team Santos Laguna in season 2023
Merging datasets...
Retrieving info from team Guadalajara in season 2023
Retrieving shooting info from team Guadalajara in season 2023
Merging datasets...
Retrieving info from team Leon in season 2023
Retrieving shooting info from team Leon in season

In [26]:


league_matches = pd.concat(all_matches)

In [27]:
league_matches

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,team_name,season
0,2022-07-03,19:05,Liga MX,Apertura 2022 Regular Season,Sun,Away,L,3.0,4.0,Santos,...,Match Report,NaN,16,7,19.0,1.0,1,1,Monterrey,2023
1,2022-07-09,21:05,Liga MX,Apertura 2022 Regular Season,Sat,Home,W,3.0,2.0,América,...,Match Report,NaN,19,8,20.7,0.0,0,0,Monterrey,2023
2,2022-07-17,17:00,Liga MX,Apertura 2022 Regular Season,Sun,Away,W,1.0,0.0,Atlético,...,Match Report,NaN,18,4,24.1,0.0,1,1,Monterrey,2023
3,2022-07-21,21:00,Liga MX,Apertura 2022 Regular Season,Thu,Away,W,3.0,0.0,Querétaro,...,Match Report,NaN,15,6,16.8,1.0,0,0,Monterrey,2023
4,2022-07-26,19:00,Liga MX,Apertura 2022 Regular Season,Tue,Home,W,1.0,0.0,Puebla,...,Match Report,NaN,22,7,17.7,0.0,0,0,Monterrey,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,2018-04-15,12:00,Liga MX,Clausura 2018 Regular Season,Sun,Home,W,4,2,Puebla,...,Match Report,NaN,0,0,NaN,NaN,0,0,UNAM,2018
32,2018-04-22,18:00,Liga MX,Clausura 2018 Regular Season,Sun,Away,W,2,1,Santos,...,Match Report,NaN,0,0,NaN,NaN,0,0,UNAM,2018
33,2018-04-29,12:00,Liga MX,Clausura 2018 Regular Season,Sun,Home,D,1,1,Querétaro,...,Match Report,NaN,0,0,NaN,NaN,0,0,UNAM,2018
34,2018-05-02,19:30,Liga MX,Quarter-finals,Wed,Home,L,1,4,América,...,Match Report,Leg 1 of 2,0,0,NaN,NaN,1,1,UNAM,2018


In [28]:
league_matches.columns = [c.lower() for c in league_matches.columns] 


In [29]:
#league_matches["gf"] = league_matches["gf"].astype(float)
league_matches["gf"] = pd.to_numeric(league_matches["gf"], errors = 'coerce')
league_matches["ga"] = pd.to_numeric(league_matches["ga"], errors = 'coerce')


In [1]:
league_matches["gf"] =league_matches["gf"].astype('Int64')
league_matches["ga"] =league_matches["ga"].astype('Int64')

league_matches.head()

NameError: name 'league_matches' is not defined

In [31]:
league_matches.shape

(3754, 27)

In [36]:
league_matches = league_matches.sort_values(by=['date'])

In [37]:
league_matches.to_csv("matches23-17.csv")

In [35]:
today = datetime.date.today()

year = today.strftime("%Y")
year

'2023'